<a href="https://colab.research.google.com/github/dustinak80/Information-Visualization/blob/master/Project/Edit_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/Info Visualization/World_Bank_Data2.csv')

In [0]:
data.head()

,Country Name,Country Code,Series Name,Series Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,China,CHN,GDP per person employed (constant 1990 PPP $),SL.GDP.PCAP.EM.KD,6553.757813,7053.375977,7624.646973,8304.691406,9067.948242,10012.6123,11209.3877,12703.79297,13951.99316,15207.08398,16777.57617,18326.94336,19730.35547,21230.38086,22763.43359
1,China,CHN,"Government expenditure on education, total (% ...",SE.XPD.TOTL.GD.ZS,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2,China,CHN,Access to electricity (% of population),EG.ELC.ACCS.ZS,96.243927,96.55620575,96.85614014,97.1497879,97.44321442,97.74249268,98.06681061,98.39870453,98.74991608,99.11564636,99.7,99.74872589,99.91638184,99.98311615,99.99848938
3,China,CHN,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,992.9433847,1076.549121,1194.856295,1379.484632,1585.838782,1782.312153,2039.01466,2325.926769,2446.369055,2612.45662,2943.589954,3298.003913,3474.988062,3773.405384,3927.0445
4,China,CHN,Fixed broadband Internet subscribers (per 100 ...,IT.NET.BBND.P2,0.001765899,0.026049283,0.253994432,0.858809836,1.897982349,2.826069624,3.825808914,4.968130974,6.16468769,7.690293166,9.291158372,11.44345583,12.7387417,13.66140637,14.42209207


In [0]:
data.tail(n = 6)

,Country Name,Country Code,Series Name,Series Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
683,Portugal,PRT,"School enrollment, tertiary (% gross)",SE.TER.ENRR,47.63357925,50.37001038,52.61494827,54.5274086,55.31507111,55.08795166,55.05118942,57.43133926,61.22639084,62.25696182,65.20931244,68.03363037,68.12140656,65.76940918,65.14974213
684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687,Data from database: Jobs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688,Last Updated: 04/09/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#drop the last 5 rows because they are nan and information from World Bank Data
data = data.iloc[:-5,:]
data.tail()

,Country Name,Country Code,Series Name,Series Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
679,Portugal,PRT,"Contributing family workers, total (% of total...",SL.FAM.WORK.ZS,2.290999889,2.26699996,1.833999991,1.559000015,1.440999985,1.457999945,1.218999982,1.069000006,0.913999975,0.869000018,0.952000022,0.619000018,0.671000004,0.677999973,0.541999996
680,Portugal,PRT,Age dependency ratio (% of working-age populat...,SP.POP.DPND,47.79974416,47.83672955,47.97564422,48.19349337,48.4079835,48.55214635,49.16433684,49.64454509,50.01264529,50.34593841,50.71447937,51.17772615,51.68182497,52.23802679,52.82307393
681,Portugal,PRT,"School enrollment, primary (% gross)",SE.PRM.ENRR,122.3840103,122.217659,118.3535614,118.8091736,117.94133,117.4743195,117.41185,116.4422989,115.1516495,112.6606979,111.091507,110.2782364,109.1242294,109.0199127,107.6059036
682,Portugal,PRT,"School enrollment, secondary (% gross)",SE.SEC.ENRR,104.0786591,105.1287613,106.6573105,105.8210602,94.48364258,97.34256744,97.91605377,101.3819809,103.1230621,105.3452225,106.2137604,106.0726166,108.2000198,118.8957901,115.5455627
683,Portugal,PRT,"School enrollment, tertiary (% gross)",SE.TER.ENRR,47.63357925,50.37001038,52.61494827,54.5274086,55.31507111,55.08795166,55.05118942,57.43133926,61.22639084,62.25696182,65.20931244,68.03363037,68.12140656,65.76940918,65.14974213


Looking at the Table above, I want to do the following:
- Pivot all of the rows into a single column
- Pivot all of the Series Name (+Series Code or delete?) as column values

The methods used in attempting this:
- Drop the repeat columns
- set select columns as index
- Then stack the rest of the columns (years in this case)

In [0]:
# 1 - Drop the columns Country Code and Series Code
data2 = data.drop(columns = ['Country Code', 'Series Code'])
data2.columns

Index(['Country Name', 'Series Name', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014'],
      dtype='object')

In [0]:
# 2 - set the index names
data2.set_index(['Country Name', 'Series Name'], inplace = True)
data2.index

MultiIndex([(   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            (   'China', ...),
            ...
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...),
            ('Portugal', ...)],
           names=['Country Name', 'Series Name'], length=684)

In [0]:
#Now I should be able to stack it.
data2.head()

2000  ...         2014
Country Name Series Name                                                      ...             
China        GDP per person employed (constant 1990 PPP $)       6553.757813  ...  22763.43359
             Government expenditure on education, total (% o...           ..  ...           ..
             Access to electricity (% of population)               96.243927  ...  99.99848938
             Electric power consumption (kWh per capita)         992.9433847  ...    3927.0445
             Fixed broadband Internet subscribers (per 100 p...  0.001765899  ...  14.42209207

[5 rows x 15 columns]

In [0]:
data2 = data2.stack()
data2.head(n = 20)

Country Name  Series Name                                                
China         GDP per person employed (constant 1990 PPP $)          2000    6553.757813
                                                                     2001    7053.375977
                                                                     2002    7624.646973
                                                                     2003    8304.691406
                                                                     2004    9067.948242
                                                                     2005     10012.6123
                                                                     2006     11209.3877
                                                                     2007    12703.79297
                                                                     2008    13951.99316
                                                                     2009    15207.08398
                                    

In [0]:
data2 = data2.unstack('Series Name')
data2.head()

Series Name       Access to electricity (% of population)  ... Wage and salaried workers, total (% of total employment) (modeled ILO estimate)
Country Name                                               ...                                                                                
Australia    2000                                     100  ...                                        79.96399689                             
             2001                                     100  ...                                        79.73999786                             
             2002                                     100  ...                                        79.60099792                             
             2003                                     100  ...                                         80.0719986                             
             2004                                     100  ...                                        80.28500366                             

[5 rows x 36 columns]

In [0]:
data2.shape

(285, 36)

In [0]:
#See which colums are mostly 0
columns = data2.columns
len(columns)

32

In [0]:
#this turns all of the '..' values in Nan
data2 = data2[data2[columns]!='..']

In [0]:
data2.isnull().sum()/len(data2)

Series Name
Access to electricity (% of population)                                                                    0.000000
Age dependency ratio (% of working-age population)                                                         0.000000
Agriculture, value added (% of GDP)                                                                        0.024561
Contributing family workers, total (% of total employment) (modeled ILO estimate)                          0.000000
Electric power consumption (kWh per capita)                                                                0.000000
Employers, total (% of total employment) (modeled ILO estimate)                                            0.000000
Employment in agriculture (% of total employment) (modeled ILO estimate)                                   0.000000
Employment in industry (% of total employment) (modeled ILO estimate)                                      0.000000
Employment in services (% of total employment) (modeled ILO 

In [0]:
remove = ['School enrollment, tertiary (% gross)', 
          'Labor force with advanced education (% of total working-age population with advanced education)', 
          'Labor force with basic education (% of total working-age population with basic education)', 
          'Labor force with intermediate education (% of total working-age population with intermediate education)']

data2.drop(columns = remove, inplace = True)
data2.isnull().sum()/len(data2)

Series Name
Access to electricity (% of population)                                                          0.000000
Age dependency ratio (% of working-age population)                                               0.000000
Agriculture, value added (% of GDP)                                                              0.024561
Contributing family workers, total (% of total employment) (modeled ILO estimate)                0.000000
Electric power consumption (kWh per capita)                                                      0.000000
Employers, total (% of total employment) (modeled ILO estimate)                                  0.000000
Employment in agriculture (% of total employment) (modeled ILO estimate)                         0.000000
Employment in industry (% of total employment) (modeled ILO estimate)                            0.000000
Employment in services (% of total employment) (modeled ILO estimate)                            0.000000
Exports of goods and services (% o

In [0]:
#Export to CSV
data2.to_csv('/content/drive/My Drive/Info Visualization/World_Bank_Data_edit.csv')